# 📓 Amazon SP-API Sales Fetcher
### This notebook connects to Amazon's Selling Partner API (SP-API)
### and retrieves your recent sales/orders in a safe, TOS-compliant way.

# Step 0: Prerequisites

Before running this notebook, complete these steps **in order**:

1. **Amazon Seller Central Account**  
   - Must be a **Professional Seller Account**.

2. **Register as a Developer** in Seller Central  
   - Go to: Apps & Services → Developer Central → "Register as Developer".

3. **Create a Private App**  
   - Name: e.g., `MySalesFetcher`  
   - Get the **Client ID** and **Client Secret**.

4. **AWS IAM Credentials**  
   - Create an IAM User with `AmazonSPAPIAccess` policy.  
   - Get **AWS Access Key** and **Secret Key**.

5. **Login With Amazon (LWA) Credentials**  
   - From your app, note **LWA Client ID** and **LWA Client Secret**.

6. **Authorize the App**  
   - In Seller Central → Manage Your Apps → Authorize.
   - Get the **Refresh Token**.

Now you’re ready to run this notebook.

# Step 1: Install Required Libraries

We need:
- **sp-api** → Amazon SP-API Python wrapper
- **python-dotenv** → to securely store credentials
- **pandas** → to save orders to CSV

Run the cell below.

In [ ]:
!pip install python-amazon-sp-api python-dotenv pandas

# Step 2: Create and Configure Your `.env` File

We’ll store credentials in a `.env` file to avoid hardcoding sensitive info.  
Replace the placeholders in the cell below with your actual credentials.

In [ ]:
env_content = """
LWA_CLIENT_ID=your_lwa_client_id
LWA_CLIENT_SECRET=your_lwa_client_secret
LWA_REFRESH_TOKEN=your_refresh_token
AWS_ACCESS_KEY=your_aws_access_key
AWS_SECRET_KEY=your_aws_secret_key
AWS_ROLE_ARN=your_aws_role_arn
"""

with open(".env", "w") as f:
    f.write(env_content)

print("✅ .env file created! Replace the placeholder values with your real credentials.")

# Step 3: Load Credentials and Initialize API Connection

We’ll read credentials from `.env` and set up the `Orders` API object.

In [ ]:
from sp_api.api import Orders
from sp_api.base import Marketplaces
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

lwa_client_id = os.getenv("LWA_CLIENT_ID")
lwa_client_secret = os.getenv("LWA_CLIENT_SECRET")
refresh_token = os.getenv("LWA_REFRESH_TOKEN")
aws_access_key = os.getenv("AWS_ACCESS_KEY")
aws_secret_key = os.getenv("AWS_SECRET_KEY")
role_arn = os.getenv("AWS_ROLE_ARN")

orders_api = Orders(
    marketplace=Marketplaces.US,
    refresh_token=refresh_token,
    lwa_app_id=lwa_client_id,
    lwa_client_secret=lwa_client_secret,
    aws_access_key=aws_access_key,
    aws_secret_key=aws_secret_key,
    role_arn=role_arn
)

print("✅ API connection initialized!")


# Step 4: Fetch Recent Orders

This example fetches all orders from the **last 7 days**.

In [ ]:
from datetime import datetime, timedelta

created_after = (datetime.utcnow() - timedelta(days=7)).isoformat() + "Z"

try:
    res = orders_api.get_orders(CreatedAfter=created_after)
    orders = res.payload.get("Orders", [])
    
    if orders:
        for order in orders:
            print("Order ID:", order["AmazonOrderId"])
            print("Purchase Date:", order["PurchaseDate"])
            print("Order Status:", order["OrderStatus"])
            print("Order Total:", order.get("OrderTotal", {}).get("Amount", "N/A"))
            print("-" * 40)
    else:
        print("No orders found in the last 7 days.")
except Exception as e:
    print("❌ Error fetching orders:", e)

# Step 5: Save Orders to CSV

We’ll save the fetched orders to a file named `amazon_orders.csv` for later analysis.

In [ ]:
import pandas as pd

if orders:
    df = pd.DataFrame(orders)
    df.to_csv("amazon_orders.csv", index=False)
    print(f"✅ Orders saved to amazon_orders.csv ({len(df)} rows).")
else:
    print("No data to save.")